In [1]:
from underdogcowboy import adm, compress_tester, type_maker, Agent

from typing import Any

class RefinementPattern:
    def __init__(self, base_agent, refiner_agent, assessor_agent):
        self.base_agent = base_agent
        self.refiner_agent = refiner_agent
        self.assessor_agent = assessor_agent
        self.best_result = None
        self.best_score = float('-inf')

    def refine(self, input_data, max_iterations=5):
        current_result = self.base_agent >> input_data
        
        for _ in range(max_iterations):
            refined_result = self.refiner_agent >> current_result
            
            base_score = self._assess(current_result)
            refined_score = self._assess(refined_result)
            
            if refined_score > base_score:
                current_result = refined_result
                if refined_score > self.best_score:
                    self.best_result = refined_result
                    self.best_score = refined_score
            else:
                # Provide feedback to refiner_agent
                self.refiner_agent >> f"Your refinement didn't improve the result. Original score: {base_score}, Your score: {refined_score}"
            
            if refined_score >= 0.95:  # Assuming a perfect score is 1.0
                break
        
        return self.best_result

    def _assess(self, result: Any) -> float:
        assessment_request = f"Please assess the following result and provide a score between 0 and 1:\n\n{result}"
        assessment_response = self.assessor_agent >> assessment_request
        
        # Extract the score from the assessment_response
        # This might need to be adjusted based on the actual format of the assessor's response
        try:
            score = float(assessment_response.split()[-1])
            return min(max(score, 0), 1)  # Ensure the score is between 0 and 1
        except ValueError:
            print(f"Warning: Could not extract a valid score from the assessor's response: {assessment_response}")
            return 0.0

# Usage example
assessor_agent = Agent("assessor.json", "agents")  # Assuming we have an assessor agent defined

type_maker_refinement = RefinementPattern(
    base_agent=type_maker,
    refiner_agent=compress_tester,
    assessor_agent=assessor_agent
)

result = type_maker_refinement.refine("file {file_ref}")
print(result)









DEBUG:underdogcowboy:Basic imports completed
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:underdogcowboy:About to load agents
DEBUG:underdogcowboy:Loading agents from package: underdogcowboy.agents
DEBUG:underdogcowboy:Creating agent for new_language.json
DEBUG:underdogcowboy:Creating agent for compress_tester.json
DEBUG:underdogcowboy:Creating agent for type_maker.json
DEBUG:underdogcowboy:Creating agent for assessment_tester.json
DEBUG:underdogcowboy:Creating agent for commit_promo.json
DEBUG:underdogcowboy:Creating agent for gitbookdoc.json
DEBUG:underdogcowboy:Creating agent for test_agent.json
DEBUG:underdogcowboy:Creating agent for testrene.json
DEBUG:underdogcowboy:Creating agent for test_agent_04.json
DEBUG:underdogcowboy:Creating agent for test_agent_03.json
DEBUG:underdogcowboy:Creating agent for test_agent_02.json
DEBUG:underdogcowboy:Loaded 11 agents
DEBUG:underdogcowboy:Adding agents to namespace
DEBUG:underdogcowboy:Adding agent new_language to namespace
DEBU

Creating AgentDialogManager
Dummy output prompt


In [ ]:
from typing import Optional, Dict, Any, Union, Callable, List
from abc import ABC, abstractmethod

class Capability:
    def __init__(self, name: str, method: Callable, input_type: type, output_type: type):
        self.name = name
        self.method = method
        self.input_type = input_type
        self.output_type = output_type

class Agent(ABC):
    def __init__(self, name: str):
        self.name = name
        self.capabilities: List[Capability] = []
        self.register_capabilities()

    @abstractmethod
    def register_capabilities(self):
        pass

    def add_capability(self, name: str, method: Callable, input_type: type, output_type: type):
        self.capabilities.append(Capability(name, method, input_type, output_type))

    def get_capability(self, name: str) -> Optional[Capability]:
        return next((cap for cap in self.capabilities if cap.name == name), None)

    def __or__(self, other: 'Agent') -> 'ChainedAgent':
        return ChainedAgent([self, other])

    def __rshift__(self, message: str) -> Any:
        capability = next((cap for cap in self.capabilities if cap.input_type == str), None)
        if capability:
            return capability.method(self, message)
        else:
            raise TypeError(f"{self.name} cannot handle string input")

    def process(self, input_data: Any) -> Any:
        for capability in self.capabilities:
            if isinstance(input_data, capability.input_type):
                return capability.method(self, input_data)
        raise TypeError(f"{self.name} cannot process input of type {type(input_data)}")

class ChainedAgent(Agent):
    def __init__(self, agents: List[Agent]):
        super().__init__("ChainedAgent")
        self.agents = agents

    def register_capabilities(self):
        pass  # ChainedAgent doesn't have its own capabilities

    def process(self, input_data: Any) -> Any:
        result = input_data
        for agent in self.agents:
            result = agent.process(result)
        return result

    def __or__(self, other: Agent) -> 'ChainedAgent':
        return ChainedAgent(self.agents + [other])

# Example specialized agents
class TypeSetterAgent(Agent):
    def register_capabilities(self):
        self.add_capability("typeset", self.typeset, str, str)

    def typeset(self, text: str) -> str:
        return f"Typeset: {text}"

class CompressAgent(Agent):
    def register_capabilities(self):
        self.add_capability("compress", self.compress, str, str)

    def compress(self, text: str) -> str:
        return f"Compressed: {text[:10]}..."

class AssessAgent(Agent):
    def register_capabilities(self):
        self.add_capability("assess", self.assess, str, dict)

    def assess(self, text: str) -> dict:
        return {"assessment": "Good", "score": 0.8}

# Create instances of our agents
type_maker = TypeSetterAgent("TypeSetter")
compress_tester = CompressAgent("Compressor")
assess = AssessAgent("Assessor")

# DialogManager (simplified for this example)
class DialogManager:
    def prepare_agent(self, agent):
        pass  # In a real implementation, this would prepare the agent

adm = DialogManager()